In [1]:
using Pkg
Pkg.activate("EnvFerrite")

  Activating project at `~/Code/Julia/FerriteStuff/Notebooks/Github/EnvFerrite`


# Electrical Impedance Tomography with TV Regularization

## Introduction

In this example we give a basic implementation of of the real valued Calderon problem relevant to Electrical Impedance Tomography. We will generate data and afterwards solve the inverse problem with a numerical solver and implement TV regularization.  

### Forward EIT:
Given a conductivity $\gamma: \Omega\subset\mathbb{R}^2 \rightarrow \mathbb{R}_{+}$ our solution $u\in H^1(\Omega,\mathbb{R}^2)$ (2D case) has to confirm to the equation:


$$ \nabla \cdot(\gamma\nabla u) = 0 \quad \forall x \in \Omega $$

For that equation we will choose a set of electrical current patterns 

$$ g_1, ..., g_n \in H^{-\frac{1}{2}}(\partial \Omega, \mathbb{R}) $$

such that:  

$$ \int_{\partial\Omega}g_i \, d\partial\Omega = 0 $$

to inject into the material via Neumann boundary conditions:

$$ \gamma\frac{\partial u_i}{\partial n} =g_i \quad \forall x \in \partial\Omega $$

In order to get the corresponding voltages 
$$ f_1, ..., f_n \in H^{\frac{1}{2}}(\partial \Omega, \mathbb{R}) $$
we will measure the corresponding voltage as:
$$ f_i := u_i|_{\partial\Omega} $$

with those boundary pairs $ (f_1,g_1), ... , (f_n,g_n) $ we now have an approximation of the Dirichlet to Neumann map:
$$ \Lambda_\gamma: H^{\frac{1}{2}}(\partial \Omega)\rightarrow H^{-\frac{1}{2}}(\partial \Omega) $$
This is called forward EIT since we just approximated the map:
$$ \gamma \rightarrow  \Lambda_\gamma $$

However real Electrical Impedance Tomography requires us to solve an **inverse Problem** where we have to reconstruct:
$$ \Lambda_\gamma \rightarrow  \gamma $$
for our approximation of $\Lambda_\gamma$ given by voltage-current boundary pairs $(f_i,g_i)$.

### Weak formulation
Given the strong formulation:
$$ \nabla \cdot(\gamma\nabla u) = 0 \quad \text{with Neumann BC:}\quad \gamma\frac{\partial u}{\partial n} = g $$

The weak formulation is:
$$ \int\limits_\Omega \gamma \nabla(u)\cdot\nabla(v) \, d \Omega = \int\limits_{\partial\Omega} g \,v \,d\partial\Omega  \quad \forall v\in H^1(\Omega)$$

### Inverse EIT
We will use $\gamma$ to refer to the true underlying conductivity and $\sigma$ for our current conductivity guess.
We will choose the simplest minimization functional for optimization:
$$ J_i(u_i,\sigma) = \| f_i- u_i\|^2_{\mathcal{L}^2(\partial\Omega)} = \int_{\partial\Omega}(f_i-u_i)^2 \,d\partial\Omega $$
Such that our problem becomes:
$$ \underset{\sigma}{\min} \sum\limits_{i=1}^n J_i(u_i,\sigma) $$
such that:
$$ \nabla u_i\cdot(\sigma\nabla u_i) = 0\quad  \text{and Neumann BC}\quad \sigma\frac{\partial u_i}{\partial n} = g_i \quad \forall i\in\{1, ...,n\}$$
Given the problem our lagrangian becomes:
$$ \mathcal{L}(\sigma, u, \lambda) = \sum\limits_{i=1}^n\left( J_i(u_i,\sigma) + \langle \lambda_i, \nabla\cdot(\sigma\nabla u_i) \rangle_{\mathcal{L}^2(\Omega)}   \right) $$

from this we will use [Adjoint state methods](https://en.wikipedia.org/wiki/Adjoint_state_method#General_case) to calculate the gradient.
Without stating any of the steps of the derivation we end up with:
- State Equation (Variation with $\delta_\lambda$)
$$ \nabla \cdot(\sigma\nabla u_i) = 0\quad  \text{with Neumann BC}\quad \sigma\frac{\partial u_i}{\partial n} = g_i$$

- Adjoint Equation (Variation with $\delta_u$)
$$ \nabla \cdot(\sigma\nabla\lambda_i) = 0\quad  \text{with Neumann BC}\quad \sigma\frac{\partial \lambda_i}{\partial n} = 2(u_i-f_i)$$
- Functional Derivative (Variation with $\delta_\sigma$)
$$ \delta_i\sigma = -\nabla u_i \cdot \nabla \lambda_i $$

#### Total Variation (TV) regularization
for real world examples and numerical stability we have to assume that our system contains some noise, like $f = f_{true} + \epsilon$. Since the inverse EIT problem is highly ill conditioned we have to consider regularization.


Because $|\nabla\sigma|^2$ is non-differentiable when $\nabla\sigma=0$, we use
$$
\mathcal{R}_{TV}(\sigma) = \int_\Omega \sqrt{|\nabla\sigma|^2+\eta}\, d\Omega,
$$
and the gradient:
$$ \delta_\sigma\mathcal{R}= -\nabla\cdot \left(   \frac{\nabla\sigma}{\sqrt{|\nabla \sigma|^2+ \eta}} \right) $$
with a small $\eta$ to revent division by zero.
This is a $L^2$ projection that requires us to sove the weak form.
##### Weak formulation of TV Regularizer:
$$ \int_\Omega wv \,d\Omega = \int_\Omega \frac{\nabla(\sigma)}{\sqrt{|\nabla\sigma|^2+\eta}}\cdot\nabla(v)\, d\Omega \quad \forall v\in FESpace$$


### Full reconstruction Algorithm (Conceptual)
+ Preallocate Massmatrix M and L^2 projector (Cholesky factorization)
+ Start with conductivity guess $\sigma_0$ (In our case: $\sigma_0(x) = 1.0$)
+ Preallocate & initialize Conjugate Gradient(CG) solver for $u_1, ...,u_n.\lambda_1,.., \lambda_n,w$.
+ Repeat till tolerance is reached or other stopping condition:
    + From $\sigma_t$ assemble stiffness matrix $K_{\sigma_t}$
    + for all $i = 1, ...,n$ (in parallel)
        + Calculate $u_i$ (State equation) as well as the $\mathcal{L}^2(\partial\Omega)$ error: $\delta u_i$
        + Calculate $\lambda_i$ (Adjoint equation)
        + Calculate $\delta\sigma_i$ (Functional derivative)
    + Calculate TV Regularization gradient and error.
    + Update $\sigma_{t+1} = \sigma_t +\beta\, \delta_{TV}\sigma + \sum_{i=1}^n \alpha_i\,\delta_i\sigma $ (with Gauss-Newton with Levenberg-Marquardt).

## Implementation
### Preliminaries
Obvious Imports:

In [2]:
using Ferrite
using SparseArrays
using LinearAlgebra
using Revise
using Interpolations
using Plots
using Statistics
using IterativeSolvers

For simplicity we will use a quadratic grid with quadrilateral elements. We are using Quadrilaterals for now:

In [3]:
grid = generate_grid(Quadrilateral, (6, 6));
dim = Ferrite.getspatialdim(grid)
order = 1

ip = Lagrange{RefQuadrilateral, order}()
qr = QuadratureRule{RefQuadrilateral}(2)
qr_face = FacetQuadratureRule{RefQuadrilateral}(2)
cellvalues = CellValues(qr, ip)
facetvalues = FacetValues(qr_face, ip)

dh = DofHandler(grid)
add!(dh, :u, ip)
close!(dh)



∂Ω = union(getfacetset.((grid,), ["left", "top", "right", "bottom"])...)
length(∂Ω)

24

For later use we will assemble and cholesky decompose the mass matrix once.

In [4]:
# This is supposed to be: ∫(u*v)dΩ
function assemble_M(cellvalues::CellValues,dh::DofHandler)
    M = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Me = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Me, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)      
            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    φⱼ = shape_value(cellvalues, q_point, j)
                    Me[i,j] += φᵢ * φⱼ * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Me)
    end
    return M, cholesky(M)
end   

M, MC = assemble_M(cellvalues,dh)

(sparse([1, 2, 3, 4, 1, 2, 3, 4, 5, 6  …  40, 41, 42, 47, 48, 49, 41, 42, 48, 49], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  48, 48, 48, 48, 48, 48, 49, 49, 49, 49], [0.012345679012345663, 0.006172839506172831, 0.003086419753086416, 0.006172839506172832, 0.006172839506172831, 0.024691358024691336, 0.012345679012345666, 0.003086419753086416, 0.006172839506172834, 0.0030864197530864174  …  0.00308641975308642, 0.012345679012345671, 0.0030864197530864165, 0.006172839506172841, 0.024691358024691343, 0.006172839506172832, 0.0030864197530864165, 0.006172839506172833, 0.006172839506172832, 0.012345679012345664], 49, 49), SparseArrays.CHOLMOD.Factor{Float64, Int64}
type:    LLt
method:  simplicial
maxnnz:  344
nnz:     344
success: true
)

Furthermore we want to know which entries of the force vector correspond to the boundary:
We will get:
- the count of nonzero entries in the force vector
- the position of non zero entries
- a function "up" to cast a vector of length of boundary dofs into the length of the force vector
- a function "down" to cast a vector into the length of the dofs of the force vector that lay on the boundary.

In [5]:
function produce_nonzero_positions(v, atol=1e-8, rtol=1e-5)
    approx_zero(x; atol=atol, rtol=rtol) = isapprox(x, 0; atol=atol, rtol=rtol)
    non_zero_count = count(x -> !approx_zero(x), v)
    non_zero_positions = zeros(Int, non_zero_count)
    non_zero_indices = findall(x -> !approx_zero(x), v)
    g_down = (x) -> x[non_zero_indices]
    g_up = (x) -> begin
        v = zeros(eltype(x), length(v))
        v[non_zero_indices] = x
        return v
    end
    return non_zero_count, non_zero_positions, g_down, g_up
end
function produce_nonzero_positions(facetvalues::FacetValues, dh::DofHandler, ∂Ω)
    f = zeros(ndofs(dh))
        for facet in FacetIterator(dh, ∂Ω)
        fe = zeros(ndofs_per_cell(dh))
        reinit!(facetvalues, facet)
        for q_point in 1:getnquadpoints(facetvalues)
            dΓ = getdetJdV(facetvalues, q_point)            
            for i in 1:getnbasefunctions(facetvalues)
                δu = shape_value(facetvalues, q_point, i)
                fe[i] += δu * dΓ
            end
        end
        assemble!(f, celldofs(facet), fe)
    end
     nzc, nzpos, g_down, g_up = produce_nonzero_positions(f)
     return  nzc, nzpos, g_down, g_up, f
end

nzc,nzpos, down, up = produce_nonzero_positions(facetvalues, dh,∂Ω)
@assert nzc == length(∂Ω)  # This is not true in Gridap.jl

In [6]:
## Sanity check:
# I have never questioned the assumption that up∘down == id and down∘up == id. Maybe I should check this.
test_vec = [i for i in 1:nzc]
@assert down(up(test_vec)) == test_vec
@assert up(down(up(test_vec))) == up(test_vec)

### Data generation

Now we will make up some conductivity. As well as some current patterns:

In [7]:
conductivity  = (x) -> 1.1 + sin(x[1]) * cos(x[2])

#22 (generic function with 1 method)

For later we want to project that function unto Q1 FE space for that we want to 

<span style="color:red">
I am somewhat clueless on how to actually project the function conductivity unto a vector living in the FE space.
The example as stated in the documentation https://ferrite-fem.github.io/Ferrite.jl/stable/reference/export/#Projection-of-quadrature-point-data seems to not work. 
Below I try to assemble a function projector as described in the domentation. it fails for now.
</span>

In [8]:
function assemble_function_vector(cellvalues::CellValues, dh::DofHandler, f, M)
    F = zeros(ndofs(dh))
    n_basefuncs = getnbasefunctions(cellvalues)
    Fe = zeros(n_basefuncs)
    cdofs = zeros(Int, n_basefuncs)

    for cell in CellIterator(dh)
        fill!(Fe, 0.0)
        reinit!(cellvalues, cell)
        coords = getcoordinates(cell)
        println(coords)
        cdofs = celldofs(cell)
        for q in 1:getnquadpoints(cellvalues)
            x_q = spatial_coordinate(cellvalues, q, coords)
            f_val = f(x_q)
            dΩ = getdetJdV(cellvalues, q)

            for i in 1:n_basefuncs
                Fe[i] += f_val * shape_value(cellvalues, q, i) * dΩ
            end
        end

        
        # Assemble local vector into global vector
  
        assemble!(F, cdofs,Fe)
    end
    return M \ F

end

cond_vec = assemble_function_vector(cellvalues, dh, conductivity, M)

Vec{2, Float64}[[-1.0, -1.0], [-0.6666666666666667, -1.0], [-0.6666666666666667, -0.6666666666666669], [-1.0, -0.6666666666666667]]
Vec{2, Float64}[[-0.6666666666666667, -1.0], [-0.3333333333333334, -1.0], [-0.3333333333333334, -0.6666666666666667], [-0.6666666666666667, -0.6666666666666669]]
Vec{2, Float64}[[-0.3333333333333334, -1.0], [0.0, -1.0], [0.0, -0.6666666666666667], [-0.3333333333333334, -0.6666666666666667]]
Vec{2, Float64}[[0.0, -1.0], [0.33333333333333326, -1.0], [0.33333333333333326, -0.6666666666666667], [0.0, -0.6666666666666667]]
Vec{2, Float64}[[0.33333333333333326, -1.0], [0.6666666666666667, -1.0], [0.6666666666666667, -0.6666666666666667], [0.33333333333333326, -0.6666666666666667]]
Vec{2, Float64}[[0.6666666666666667, -1.0], [1.0, -1.0], [1.0, -0.6666666666666667], [0.6666666666666667, -0.6666666666666667]]
Vec{2, Float64}[[-1.0, -0.6666666666666667], [-0.6666666666666667, -0.6666666666666669], [-0.6666666666666667, -0.3333333333333334], [-1.0, -0.333333333333333

49-element Vector{Float64}:
 0.636600296812608
 0.7592846200040305
 0.6049646371557323
 0.4267129877964103
 0.9197560099956572
 0.8381182235056315
 1.1000000000000003
 1.100000000000001
 1.280243990004341
 1.3618817764943685
 ⋮
 1.5950353628442684
 1.77328701220359
 0.7592846200040311
 0.6366002968126072
 0.9197560099956579
 1.0999999999999996
 1.280243990004343
 1.4407153799959687
 1.5633997031873925

In [ ]:
# This function assembles the stiffness matrix from a given vector.
# Hopefully this is: ∫(γ * ∇(u)⋅∇(v))dΩ 
function assemble_K(cellvalues::CellValues, dh::DofHandler, γ::AbstractVector)
    K = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(K)
    for cell in CellIterator(dh)
        fill!(Ke, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            γe = γ[celldofs(cell)] # (Edit) Could be done more efficiently by copying into preallocated array
            σ = function_value(cellvalues, q_point, γe)
            for i in 1:n_basefuncs
                ∇v = shape_gradient(cellvalues, q_point, i)
                #u = shape_value(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    ∇u = shape_gradient(cellvalues, q_point, j)
                    Ke[i, j] += σ* (∇v ⋅ ∇u) * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Ke)
    end
    return K
end

K_from_vec = assemble_K(cellvalues, dh,cond_vec)

49×49 SparseMatrixCSC{Float64, Int64} with 361 stored entries:
⎡⣿⣿⠶⠀⠀⠀⠀⣤⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠘⠃⣿⣿⣤⠀⠀⠂⣒⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠛⣿⣿⣤⠀⠐⣒⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⣤⠠⠀⠀⠛⠛⣤⠄⠐⢢⡤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠁⠘⠸⢰⢠⢀⠁⠻⣦⡈⠘⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠈⡶⣂⠈⢱⢖⡀⠉⡶⣂⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⡄⠈⠻⣦⠀⠙⢷⡄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⢫⣄⠀⠟⣥⡀⠸⢫⣄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠷⣀⡈⠻⢆⣀⠙⠷⣀⡀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠋⢶⣄⠘⠱⣦⡀⠋⢶⣄⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢣⡤⠈⠛⣤⠄⠙⠃⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠘⢷⣄⠁⠻⣦⡀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠀⠀⠈⠁⎦

In [10]:
# This is matrix assembly on a function. How do I do it if the conductivity is given as a coefficient vector for Q1 FE Space?
# Hopefully this is: ∫(γ * ∇(u)⋅∇(v))dΩ 
function assemble_K(cellvalues::CellValues, dh::DofHandler, γ)
    K = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(K)
    for cell in CellIterator(dh)
        fill!(Ke, 0)
        reinit!(cellvalues, cell)
        for q in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q)
            x = spatial_coordinate(cellvalues, q, getcoordinates(cell))
            σ = γ(x)
            for i in 1:n_basefuncs
                ∇v = shape_gradient(cellvalues, q, i)
                for j in 1:n_basefuncs
                    ∇u = shape_gradient(cellvalues, q, j)
                    Ke[i, j] += σ * (∇v ⋅ ∇u) * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Ke)
    end
    return K
end

K_true = assemble_K(cellvalues, dh, conductivity)
K_true_LU = lu(K_true)

SparseArrays.UMFPACK.UmfpackLU{Float64, Int64}
L factor:
49×49 SparseMatrixCSC{Float64, Int64} with 344 stored entries:
⎡⠵⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠷⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⢳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠱⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⡛⠿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠑⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⢾⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢠⣤⣤⡌⠀⣬⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⢀⠀⢶⠀⠀⠀⠉⠱⠀⠰⠾⠿⠿⢷⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡄⣤⣤⡜⠀⠈⠀⠀⠀⠀⠂⠀⠀⠀⠙⠛⠛⠛⠓⣄⠀⠀⠀⠀⠀⎥
⎢⢳⡇⠀⠹⣀⣠⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⣼⣿⣿⣷⣄⠀⠀⠀⎥
⎢⠈⠁⠰⣆⠉⠉⠆⠀⠶⢶⡆⠀⠀⠀⠀⣶⣾⣿⣿⣿⣿⣿⣷⣄⠀⎥
⎣⠀⠀⠀⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠉⠉⠉⠉⠁⎦
U factor:
49×49 SparseMatrixCSC{Float64, Int64} with 344 stored entries:
⎡⠑⢇⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣭⠽⠶⠆⠀⠀⎤
⎢⠀⠀⠙⢇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⠿⣄⡀⠰⢦⡄⎥
⎢⠀⠀⠀⠀⠙⢶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⣄⡀⠀⠀⣸⡇⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠑⢦⣿⡌⠀⠀⠀⠀⠀⣀⠀⠀⠀⠀⠀⠀⠈⠁⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⡆⠀⠀⠀⠀⣿⡄⠀⠀⠀⠀⠀⢸⣇⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢤⣰⣆⠂⠉⠑⠂⠈⠀⠀⠀⠈⠉⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣦⣿⣰⡆⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⡇⣷⠀⠀⠀⢠⣤⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣿⠀⣀⣶⣾⣿⡀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢤⣿⣿⣿⣿⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⡇⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⎦

In [11]:
# Sanity check: (positive semidefinite self adjoint stiffness matrix)
@assert K_true == K_true'  # Not true in Gridap.jl
if ndofs(dh) < 500
    K_dense = Matrix(K_true)
    eig_min = minimum(eigvals(K_dense))
    @assert eig_min > -1e-14
    print("Smallest eigenvalue: ", eig_min)
end

Smallest eigenvalue: -4.18205629053751e-16

Now we generate current patterns: We assume one current source and one current sink. Important is that it sums of to zero.
We generate the right hand side force vectors $g_1, ... g_n$ as an Matrix G and calculate $f_1, ..., f_n$

In [12]:
num_modes = Int64((nzc^2-nzc)//2)
G_small = zeros(nzc,num_modes)
G = zeros(ndofs(dh),num_modes)
k = 1
for i in 1:(nzc-1)
    for j in i+1:nzc
        G_small[i,k] = 1.0
        G_small[j,k] = -1.0
        G[:,k] = up(G_small[:,k])
        k += 1
    end
end
F_big = K_true_LU \ G
F = zeros(nzc,num_modes)
k = 1
for i in 1:(nzc-1)
    for j in i+1:nzc
        F[:,k] = down(F_big[:,k])
        k += 1
    end
end
col_means = mean(F, dims=1)
F .-= col_means

24×276 Matrix{Float64}:
  2.32357     2.27301     2.84635    3.11468    …   0.0315516     0.0206735
 -0.778222    0.0544728   0.379085   0.67472        0.0290574     0.0200595
  0.105038   -1.18908     0.594049   0.821915       0.0343652     0.0213559
 -0.143693    0.0207049  -0.883317  -0.0301557      0.0227147     0.0184754
 -0.116386   -0.0197557  -0.298483  -0.940736       0.0142851     0.0163294
 -0.100137   -0.0310713  -0.23988   -0.413997   …   0.00590317    0.0141512
 -0.0951501  -0.0355859  -0.215654  -0.353677      -0.000208784   0.0125401
 -0.0938456  -0.0367458  -0.210128  -0.336535      -0.00240451    0.0119555
 -0.0928123  -0.0377007  -0.205694  -0.324858      -0.00487321    0.0112946
  0.0325847  -0.176756    0.162399   0.306009       0.0449049     0.0238275
  ⋮                                             ⋱                 ⋮
 -0.0668455  -0.0669617  -0.111387  -0.1282     …   0.105921      0.0371156
 -0.082988   -0.0473062  -0.166484  -0.230807      -0.224877     -0.0301

To be realistic we will add some noise:
We have to ensure that our noise has mean zero.

In [13]:
function mean_zero_noise(n::Int64, σ::Float64)
    out = σ * randn(n)
    mean = Statistics.mean(out)
    out .- mean
end

mean_zero_noise (generic function with 1 method)

To simplify things we can also do SVD:

In [14]:
# Implement SVD here later
# reduce the number of modes according to used SVD modes
function do_svd(F,G)
    Λ = F * G'
    num_modes = (size(Λ, 1) - 1)
    V,Σ,U = svd(Λ)
    U = U[:,1:num_modes]
    V = V[:,1:num_modes]
    col_means = mean(U, dims=1)
    U .-= col_means
    col_means = mean(V, dims=1)
    V .-= col_means
    # Not unimportant:
    V = V * diagm(Σ[1:num_modes]) 
    return V, Σ, U, Λ, num_modes
end
F, Σ, G_small, Λ, num_modes = do_svd(F,G_small)
# Apply singular Values:
G = zeros(ndofs(dh),num_modes)
F_big = copy(G)
for i in 1:num_modes
    G[:,i] = up(G_small[:,i])
    F_big[:,i] = up(F[:,i])
end

now that we have done SVD on the original choice of modes we have 
- Truncation of SVD modes, thus regularization
- Averaging of noise over multiple measurements
- Elimination of any unncessary number of nodes we choose before.

Here we define a struct where we save and preallocate all the necessary information for the solver step.

In [ ]:
# Implement a sanity check if the two matrices assembled from the function and the vector are roughly the same (use relatively coarse ≈ )
Matrix_norm = norm(K_from_vec - K_true)
println("Norm of Matrix difference: ",Matrix_norm)
@assert Matrix_norm < 20.0

g_test = G[:,1]
f_test_true = K_true \ g_test
f_test_vec = K_from_vec \g_test
vector_norm = norm(f_test_true - f_test_vec)
println("norm of difference of first SVD mode: " ,vector_norm)
@assert vector_norm < 10.0

Norm of Matrix difference: 10.449909767217584
norm of difference of first SVD mode: 9.231159780743125


In [16]:
mutable struct EITMode
    u::AbstractVector
    λ::AbstractVector
    δσ::AbstractVector
    f::AbstractVector
    g::AbstractVector
    rhs::AbstractVector
    error::Float64
    length::Int64
    m::Int64
end
function EITMode(g::AbstractVector, f::AbstractVector)
    L = length(g)
    M = length(f)
    return EITMode(zeros(L), zeros(L), zeros(L), f, g, zeros(L), 0.0, L, M)
end
mode_dict = Dict{Int64,EITMode}()
for i in 1:num_modes
    mode_dict[i] = EITMode(G[:,i],F[:,i])
end

### Solving EIT

We will now assume a starting conductivity guess $\sigma_0(x) = 1.0 $

In [17]:
σ₀ = (x) -> 1.0

#24 (generic function with 1 method)

We would prefer to save $\sigma$ as a vector for use in FEM and also have a method to export each

In [18]:
# Project function here: 
#return vector.
σ = ones(ndofs(dh))

49-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

A prerequisite is that we can calculate the bilinear map: $\nabla(u)\cdot\nabla(\lambda)$


<span style="color:red">
Here is where I struggle:
</span>

In [19]:
function calculate_bilinear_map_rhs(a::AbstractVector, b::AbstractVector, cellvalues::CellValues, dh::DofHandler)
    n = ndofs(dh)
    r = zeros(n)
    n_basefuncs = getnbasefunctions(cellvalues)
    
    # Get the type for the gradient, e.g., Vec{2, Float64} for 2D
    # This is often available from the cellvalues or interpolation
    first_grad = shape_gradient(cellvalues, 1, 1)
    grad_type = typeof(first_grad)

    for cell in CellIterator(dh)
        reinit!(cellvalues, cell)
        dofs = celldofs(cell)
        ae = a[dofs]  # Local nodal values for λ
        be = b[dofs]  # Local nodal values for u
        re = zeros(length(dofs))

        for q in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q)
            
            # --- Step 1: Calculate the full gradients at quadrature point q ---
            grad_a_at_q = zero(grad_type)
            grad_b_at_q = zero(grad_type)
            for i in 1:n_basefuncs
                ∇ϕᵢ = shape_gradient(cellvalues, q, i)
                grad_a_at_q += ae[i] * ∇ϕᵢ
                grad_b_at_q += be[i] * ∇ϕᵢ
            end
            
            # --- Step 2: Compute the dot product ---
            dot_product = grad_a_at_q ⋅ grad_b_at_q
            
            # --- Step 3: Project the result onto each test function ---
            for k in 1:n_basefuncs
                ϕₖ = shape_value(cellvalues, q, k)
                re[k] += dot_product * ϕₖ * dΩ
            end                    
        end
        assemble!(r, celldofs(cell), re)
    end
    return r
end

function calculate_bilinear_map_rhs!(mode::EITMode, cellvalues::CellValues, dh::DofHandler)
    a = mode.u
    b = mode.λ
    mode.rhs = calculate_bilinear_map_rhs(a,b,cellvalues,dh)
end

calculate_bilinear_map_rhs! (generic function with 1 method)

In [21]:
# Assemble right-hand side for the projection of ∇(u) ⋅ ∇(λ) onto the FE space.
# This computes rhs_i = ∫ (∇u ⋅ ∇λ) ϕ_i dΩ for each test function ϕ_i.
# Assuming u and λ are scalar fields in the same FE space.
# cellvalues should be CellScalarValues(qr, ip) where qr is QuadratureRule, ip is Interpolation.
function calculate_bilinear_form_rhs!(mode::EITMode, dh::DofHandler, cellvalues::CellValues)
    n = ndofs(dh)
    mode.rhs = zeros(n)  # Assuming mode.rhs is a field to store the RHS vector

    a = mode.λ  # Assuming λ is a vector of nodal values for the entire domain
    b = mode.u  # Assuming u is a vector of nodal values for the entire domain

    n_basefuncs = getnbasefunctions(cellvalues)
    for cell in CellIterator(dh)
        reinit!(cellvalues, cell)
        dofs = celldofs(cell)
        ae = a[dofs]  # Local nodal values for λ
        be = b[dofs]  # Local nodal values for u

        re = zeros(length(dofs))
        

        # Quadrature loop
        for q in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q)

            # Compute ∇λ and ∇u at the quadrature point using built-in functions
            ∇a_q = [ ae[i] * shape_gradient(cellvalues, q, i) for i in 1:n_basefuncs]
            ∇b_q = [be[i] * shape_gradient(cellvalues, q, i) for i in 1:n_basefuncs]
            
            qval = dot(∇a_q, ∇b_q)

            # Loop over test functions
            for i in 1:n_basefuncs
                ϕ_i = shape_value(cellvalues, q, i)
                re[i] += qval * ϕ_i * dΩ
            end
        end

        assemble!(mode.rhs, dofs, re)
    end
end

# To compute the projection δ such that M δ = rhs, where M is the mass matrix.
# Assuming M is pre-assembled (e.g., as a sparse matrix).
# You can assemble M separately using a similar loop for ∫ ϕ_i ϕ_j dΩ.
# Then solve using \ (which uses Cholesky for symmetric positive definite matrices if applicable).
function calculate_bilinear_map!(mode, dh::DofHandler,cellvalues::CellValues, M)
    mode.δσ = calculate_bilinear_map_rhs!(mode, cellvalues, dh)
    #mode.δσ = M \ mode.rhs
end

calculate_bilinear_map! (generic function with 1 method)

With the given matrix and projector our we need to optimize for every mode $(f_i,g_i)$:

In [22]:
function state_adjoint_step!(mode::EITMode, K::AbstractMatrix, M, dh::DofHandler, cellvalues::CellValues, maxiter=500)
    cg!(mode.u,K, mode.g; maxiter = maxiter)
    b = down(mode.u)
    b = 2*(b - mode.f)
    b .-= Statistics.mean(b)
    cg!(mode.λ, K, up(b); maxiter = maxiter)
    mode.error = norm(b)^2
    # add calculation of ∇(u)⋅∇(λ) here once figured out
    calculate_bilinear_map!(mode, dh, cellvalues, M)    
    # Check whether this needs + or - as a sign.
end

state_adjoint_step! (generic function with 2 methods)

Additinal we need to assemble the TV regularizer. The required Mass matrix we already have asembled and ready to use. 


In [23]:
mutable struct TV
    δ::AbstractArray # Is supposed to hold the error
    rhs::AbstractArray # 
    err_vec::AbstractArray
    error::Float64
end
function TV(n::Int64)
    TV(zeros(n),zeros(n),zeros(n),0.0)
end

function calc_tv_step!(σ::AbstractVector{Float64}, tv::TV, dh::DofHandler, cellvalues::CellValues, M, η=1e-8)
    n = ndofs(dh)
    r = tv.rhs
    r = zeros(n)
    tv_error = 0.0

    for cell in CellIterator(dh)
        reinit!(cellvalues, cell)
        dofs = celldofs(cell)
        σe = σ[dofs]

        re = zeros(length(dofs))
        # quadrature loop
        for q in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q)
            ∇ϕ = shape_gradient(cellvalues, q)   # size (dim, ndofs)
            ∇σ = ∇ϕ * σe                        # vector of length dim
            a = 1 / sqrt(dot(∇σ, ∇σ) + η)
            tv_error += sqrt(dot(∇σ, ∇σ) + η) * dΩ

            ϕvals = shape_function_values(cellvalues, q)
            for i in 1:length(dofs)
                ∇ϕi = ∇ϕ[:, i]
                re[i] += a * dot(∇σ, ∇ϕi) * dΩ
            end
        end
        assemble!(r, dofs, re)
    end

    tv.δ = M \ r          
    tv.error = tv_error
end
tv = TV(ndofs(dh))

TV([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0)

For finding suitable stepsizes we will use Gauss-Newton.
Since we want to avoid implementing a dense Hessian Matrix we use SVD to invert the Matrix.

In [24]:
function gauss_newton(J::Matrix{Float64}, r::Vector{Float64}; λ::Float64=1e-3)
    U, Σ, V = svd(J, full=false)
    n = length(Σ)
    Σ_damped = zeros(n)
    for i in 1:n
        Σ_damped[i] = Σ[i] / (Σ[i]^2 + λ) # Levenberg-Marquardt regularization
    end
    V * (Σ_damped .* (U' * r))
end
# Note: If we do SVD further up on the boundary pairs that we selected we can use truncated SVD to approximate/regularize the boundary operator Λ. Due to that (i.e. a highly reduced number of boundary pairs) SVD in Gauss-newton might become an efficient option. Else I want to avoid assembling a huge dense Hessian Matrix.

gauss_newton (generic function with 1 method)

Now that we have all the pieces we can assemble the full optimization step:

In [25]:
# note: If you want to use truncated SVD as regularization one can pass a smaller number than num_modes
function full_step!(M,σ::AbstractVector ,modes::Dict{Int64,EITMode}, num_modes::Int64,tv::TV, dh::DofHandler, cellvalues::CellValues, do_TV::Bool =true, β::Float64 = 1e-5)
    # Assemble Matrix: (from vector)
    K = assemble_K(cellvalues,dh,σ)
    if do_TV
        J = zeros(num_modes+1,ndofs(dh))
        r = zeros(num_modes+1)
        # Launch TV regularizer:
        tv_task = Threads.@spawn begin
            calc_TV_step!(σ,tv, dh,cellvalues,M)
        end
    else
        J = zeros(num_modes,ndofs(dh))
        r = zeros(num_modes)
    end
    # solve adjoint state method
    Threads.@threads for i in 1:num_modes
        state_adjoint_step!(mode_dict[i], K, M, dh, cellvalues)
    end

    # Fetch gradients & errors
    for i in num_modes
        J[i,:] = mode_dict[i].δσ
        r[i] = mode_dict[i].error
    end
    if do_TV
        # Fetch TV regularization
        fetch(tv_task)
        J[num_modes+1,:] = tv.δ
        r[num_modes+1] = β * tv.error 
    end    
    # calculate steps with Gauss-Newton
    δσ = gauss_newton(J, r, λ=1e-3)
    # update σ
    σ = max.(σ+δσ ,1e-12) # Ensure positivity
    return σ,δσ,r
end

full_step! (generic function with 3 methods)

Let's run this optimization loop a few times:

In [26]:
cg!(mode_dict[1].u,K, mode_dict[1].g; maxiter = 500)

UndefVarError: UndefVarError: `K` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [27]:
#=
for i in 1:10
    print("Step ", i, " ")
    result, time, bytes, gctime, memallocs = @timed begin
        full_step!(MC, σ, mode_dict, num_modes, tv, dh, cellvalues, false)
    end
    println("Time: ", time, " seconds, Bytes: ", bytes, ", GC time: ", gctime, ", Memory allocations: ", memallocs)
    println("With error: ")
end
=#

In [28]:
#σ,δσ,r = full_step!(MC, σ, mode_dict, num_modes, tv, dh, cellvalues, false)

In [29]:
# Test the above functions

## Plotting


In [30]:
# project is to grid and plot with Plots.jl
# I wanna use Plots.jl and not Makie.jl or similar because lateron i want to implement a NN on the grid as a regularizer using Lux.jl

In [31]:
# Project to dictionary (coordinate,value)
#PointEvalHandler(grid, points::AbstractVector{Vec{dim,T}})
# Put values from dictionary into Array.

This is our original and final reconstruction:

In [32]:
# Plot reconstruction and original with Plots.jl here